## Validate GPU is available for use

In [1]:
!nvidia-smi

Sun Dec 13 16:28:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    36W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import torch
torch.cuda.is_available()

True

## Install necessary packages

In [3]:
# ! pip install -U adapter-transformers

In [4]:
# ! pip install datasets

## Load and inspect data

In [5]:
import datasets
import torch
from torch.utils.data import DataLoader, TensorDataset

In [6]:
def get_dataset(dataset):
    ds = datasets.load_dataset('glue', dataset)
    num_classes = ds['train'].features['label'].num_classes
    return ds, num_classes


def create_dataset_from_text_dataset(ds, tokenizer):
    encoding = tokenizer(ds['sentence'], return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids']
    attn_masks = encoding['attention_mask']
    labels = torch.tensor(ds['label'])
    return TensorDataset(input_ids, attn_masks, labels)


def get_tensor_datasets(dataset_dict, splits, tokenizer):
    split_datasets = {}
    for s in splits:
        split_datasets[s] = create_dataset_from_text_dataset(dataset_dict[s], tokenizer)
    return split_datasets


def get_data_loaders(split_datasets, batch_size):
    train_loader = DataLoader(split_datasets['train'], batch_size, shuffle=True)
    val_loader = DataLoader(split_datasets['validation'], batch_size, shuffle=False)
    test_loader = DataLoader(split_datasets['test'], batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

In [7]:
# Load dataset
dataset = 'sst2'
print(f'Loading {dataset} dataset...')
dataset_dict, num_classes = get_dataset(dataset)

Loading sst2 dataset...


Reusing dataset glue (/home/jupyter/.cache/huggingface/datasets/glue/sst2/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [8]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [9]:
# import datasets
# sst2 = datasets.load_dataset('glue', 'sst2')
# sst2

In [10]:
# for i in range(5):
#     print(sst2['train'][i])

In [11]:
# sst2['train'].features['label'].num_classes

## Load Tokenizer

In [12]:
import torch
import transformers
from transformers import AdapterType
from transformers import BertTokenizerFast, BertForSequenceClassification


def get_tokenizer(model_name):
    if model_name == 'bert-base-uncased':
        tokenizer = BertTokenizerFast.from_pretrained(model_name)
    else:
        raise NotImplementedError

    return tokenizer


def get_transformer(model_name, num_labels, adapter, dataset):
    if model_name == 'bert-base-uncased':
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        if adapter:
            model.add_adapter(dataset, AdapterType.text_task)
            model.train_adapter(dataset)
    else:
        raise NotImplementedError

    return model


def get_criterion(num_labels):
    if num_labels == 2:
        criterion = torch.nn.CrossEntropyLoss()
    else:
        raise NotImplementedError

    return criterion

In [13]:
# Load tokenizer
model_name = 'bert-base-uncased'
print(f'Loading tokenizer for {model_name}...')
tokenizer = get_tokenizer(model_name)

Loading tokenizer for bert-base-uncased...


In [14]:
# import transformers
# from transformers import BertTokenizerFast, BertForSequenceClassification

# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizerFast.from_pretrained(model_name)

## Create data loader for various splits

In [15]:
# Create data loader for each split
splits = list(dataset_dict.keys())
print(f'Creating data loader for {splits} splits...')
split_datasets = get_tensor_datasets(dataset_dict, splits, tokenizer)
train_loader, val_loader, test_loader = get_data_loaders(split_datasets, batch_size=32)

Creating data loader for ['train', 'validation', 'test'] splits...


In [40]:
split_datasets['test']

In [43]:
dataset_dict['test']['label']

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,


In [16]:
# for i_batch, sample_batched in enumerate(train_loader):
#     print(i_batch, sample_batched[2].size())
    
#     if i_batch == 0:
#         print(sample_batched[2])
#         sb = sample_batched[2].to('cuda')
#         print(sb)
#         break

In [17]:
# from torch.utils.data import DataLoader, TensorDataset

# def create_dataset_from_text_dataset(ds):
#     encoding = tokenizer(ds['sentence'], return_tensors='pt', padding=True, truncation=True)
#     input_ids = encoding['input_ids']
#     attn_masks = encoding['attention_mask']
#     labels = torch.tensor(ds['label'])
    
#     return TensorDataset(input_ids, attn_masks, labels)

# splits = ['train',  'validation', 'test']
# split_datasets = {}

# for s in splits:
#     split_datasets[s] = create_dataset_from_text_dataset(sst2[s])

# split_datasets

In [18]:
# Validate data loader
# sample_loader = DataLoader(split_datasets['train'], batch_size=3, shuffle=True)
# for i in sample_loader:
#     input_ids, attn_masks, labels = i
#     decoded = tokenizer.batch_decode(input_ids)
#     for d in decoded:
#         print(d)
#     break

## Create model

In [19]:
# Load model
adapter = False
print(f'Loading {model_name} with adapters={adapter}...')
model = get_transformer(model_name,
                        num_labels=num_classes,
                        adapter=adapter,
                        dataset=dataset)
criterion = get_criterion(num_labels=num_classes)

Loading bert-base-uncased with adapters=False...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [20]:
# for name, child in model.named_children():
#     print(name)

In [21]:
# def create_model(add_adapters=False):
#     model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
#     if add_adapters:
#         from transformers import AdapterType
#         model.add_adapter("sst2", AdapterType.text_task)
#         model.train_adapter("sst2")
#     return model

In [22]:
# train_loader = DataLoader(split_datasets['train'], batch_size=16, shuffle=True)
# val_loader = DataLoader(split_datasets['validation'], batch_size=128, shuffle=False)

In [23]:
# from transformers import logging
# logging.set_verbosity_warning()

# model = create_model(add_adapters=True)

## Get Learning Scheme

In [24]:
import torch


def get_learning_scheme(learning_scheme, model, learning_rate, adapter, epoch):
    if learning_scheme == 'differential':
        optimizer_grouped_parameters = differential_learning_scheme(model, learning_rate)
        optimizer = torch.optim.SGD(optimizer_grouped_parameters)
    elif learning_scheme == 'fixed':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    elif learning_scheme == 'nesterov':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
    elif learning_scheme == 'gradual-unfreeze':
        optimizer_grouped_parameters = gradual_unfreezing_learning_scheme(model, learning_rate, adapter, epoch)
        optimizer = torch.optim.SGD(optimizer_grouped_parameters)
    else:
        raise NotImplementedError

    return optimizer


def differential_learning_scheme(model, learning_rate=0.1, divisor=2.6):
    param_prefixes = {}
    for n, p in model.named_parameters():
        if p.requires_grad:
            base = n.partition('.weight')[0].partition('.bias')[0]
            if base not in param_prefixes:
                param_prefixes[base] = 0

    param_prefix_divisors = list(reversed([divisor * i for i in range(1, len(param_prefixes))])) + [1]
    param_learning_rates = [learning_rate / ld for ld in param_prefix_divisors]

    param_prefix_lr_lookup = dict(zip(param_prefixes.keys(), param_learning_rates))

    optimizer_grouped_parameters = [
        {'params': p, 'lr': param_prefix_lr_lookup[n.partition('.weight')[0].partition('.bias')[0]]}
        for n, p in model.named_parameters() if p.requires_grad
    ]

    return optimizer_grouped_parameters


def gradual_unfreezing_learning_scheme(model, learning_rate, adapter, epoch=1):
    trainable_layers = []
    for n, p in model.named_parameters():
        if p.requires_grad:
            base = n.partition('.weight')[0].partition('.bias')[0]
            if adapter:
                if base not in trainable_layers and 'adapter' or 'classifier' in base:
                    trainable_layers.append(base)
            else:
                if base not in trainable_layers:
                    trainable_layers.append(base)

    optimizer_grouped_parameters = [
        {'params': p, 'lr': learning_rate}
        for n, p in model.named_parameters() if p.requires_grad and n.partition('.weight')[0].partition('.bias')[0] in trainable_layers[-epoch:]
    ]

    return optimizer_grouped_parameters


def get_scheduler(scheduler, optimizer, learning_rate, max_lr):
    if scheduler:
        if scheduler == 'cyclic-triangular':
            scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                                          base_lr=learning_rate,
                                                          max_lr=max_lr,
                                                          mode='triangular')
        else:
            raise NotImplementedError

    return scheduler

In [26]:
# Get learning scheme
learning_scheme = 'fixed'
print(f'Configuring {learning_scheme} learning scheme...')
optimizer = get_learning_scheme(learning_scheme, model, learning_rate=0.01, adapter=adapter, epoch=1)

Configuring fixed learning scheme...


In [27]:
# prefixes = {}
# for n, p in model.named_parameters():
#     base = n.partition('.weight')[0].partition('.bias')[0]
#     if base not in prefixes:
#         prefixes[base] = 0

In [28]:
# BASE_LR = 0.1
# BASE_DIVISOR = 2.6

# prefix_divisors = list(reversed([BASE_DIVISOR * i for i in range(1, len(prefixes))])) + [1]
# layer_learning_rates = [BASE_LR / ld for ld in prefix_divisors]

# prefix_lr_lookup = dict(zip(prefixes.keys(), layer_learning_rates))

In [29]:
# optimizer_grouped_parameters = [
#     {'params': p, 'lr': prefix_lr_lookup[n.partition('.weight')[0].partition('.bias')[0]]}
#     for n, p in model.named_parameters()
# ]

In [30]:
# optimizer = torch.optim.SGD(optimizer_grouped_parameters)

## Train

In [32]:
import time
import torch
import torch.nn.functional as F

In [33]:
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print('Using device:', device)
model = model.to(device)
n_epochs = 10
optimizer = optimizer
scheduler = None
criterion = criterion.to(device)

N_MINI_BATCH_CHECK = 5

def measure_performance(loader):
    running_loss = 0.0
    correct_count = 0.0
    total_count = 0.0
    for data in loader:
        input_ids = data[0].to(device)
        attn_masks = data[1].to(device)
        labels = data[2].to(device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attn_masks)[0]
            loss = criterion(outputs, labels)
            probas = F.softmax(outputs, dim=1)
            preds = torch.argmax(probas, axis=1)

            # Track stats
            running_loss += loss
            correct_count += torch.sum(preds == labels)
            total_count += len(labels)

    running_loss /= len(loader)
    acc = correct_count / total_count

    return running_loss, acc

Using device: cuda


In [38]:
for i, data in enumerate(test_loader):
    print(data)
    print(len(data))
    if i == 0:
        break

[tensor([[  101, 15491, 28616,  ...,     0,     0,     0],
        [  101,  2023,  2143,  ...,     0,     0,     0],
        [  101,  2011,  1996,  ...,     0,     0,     0],
        ...,
        [  101,  2182,  1005,  ...,     0,     0,     0],
        [  101,  2087,  2047,  ...,     0,     0,     0],
        [  101,  2061,  2261,  ...,     0,     0,     0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])]
3


In [34]:
measure_performance(test_loader)

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
if val_loader:
    print('Initial evaluating on validation dataset')
    val_loss, val_acc = measure_performance(val_loader)
    epoch_summary = f'[Epoch 0] | Val acc: {val_acc:.4f} Val loss: {val_loss:.4f}\n\n'
    print(epoch_summary)

In [ ]:
for epoch in range(n_epochs):
    print(f'--- Epoch: {epoch} ---')
    epoch_start_time = time.time()
    batch_start_time = time.time()
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        input_ids = data[0].to(device)
        attn_masks = data[1].to(device)
        labels = data[2].to(device)

        optimizer.zero_grad()

        # Evaluation/optimization step
        outputs = model(input_ids=input_ids, attention_mask=attn_masks)[0]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if scheduler:
            scheduler.step()

        # Print statistics periodically
        running_loss += loss.item()
        if i % N_MINI_BATCH_CHECK == N_MINI_BATCH_CHECK - 1:
            batch_end_time = time.time()
            total_batch_time = batch_end_time - batch_start_time

            print(
                f'[E{epoch + 1:d} B{i + 1:d}] ',
                f'Loss: {running_loss / N_MINI_BATCH_CHECK:.5f} ',
                f'Time: {total_batch_time:.2f} ',
                f'LR: {scheduler.get_last_lr()}' if scheduler else '')

            # Reset statistics
            batch_start_time = time.time()
            running_loss = 0.0

    epoch_end_time = time.time()
    total_epoch_time = epoch_end_time - epoch_start_time
    epoch_summary = '[Epoch {}] {} seconds'.format((epoch + 1), total_epoch_time)

    if val_loader:
        val_loss, val_acc = measure_performance(val_loader)
        epoch_summary += f' | Val acc: {val_acc:.4f} | Val loss: {val_loss:.4f}'

    print(epoch_summary)

print('Finished training')

In [ ]:
# import time
# import torch.nn.functional as F
# import copy

# class Trainer:
#     def __init__(self, model, n_epochs, optimizer):
#         self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#         self.model = model.to(self.device)
#         self.n_epochs = n_epochs
#         self.optimizer = optimizer
#         self.criterion = torch.nn.CrossEntropyLoss().to(self.device)

# #         no_decay = ['bias', 'LayerNorm.weight']
# #         optimizer_grouped_parameters = [
# #             {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
# #             {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# #         ]
# #         self.optimizer = torch.optim.SGD(optimizer_grouped_parameters, lr=1e-3, momentum=0.9)
#         # self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer)
#         # self.scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(self.optimizer, T_0=10000, T_mult=2)        

#     def measure_performance(self, loader):
#         running_loss = 0.0
#         correct_count = 0
#         total_count = 0
#         for data in loader:
#             input_ids = data[0].to(self.device)
#             attn_masks = data[1].to(self.device)
#             labels = data[2].to(self.device)
#             with torch.no_grad():
#                 outputs = self.model(input_ids=input_ids, attention_mask=attn_masks)[0]
#                 loss = self.criterion(outputs, labels)
#                 probas = F.softmax(outputs, dim=1)
#                 preds = torch.argmax(probas, axis=1)
                
#                 # Track stats
#                 running_loss += loss
#                 correct_count += torch.sum(preds == labels) 
#                 total_count += len(labels) 
        
#         running_loss /= len(loader)
#         acc = correct_count / total_count

#         return running_loss, acc

#     def train_loop(self, train_loader, val_loader=None):
#         print('Starting training loop\n\n')
#         N_MINI_BATCH_CHECK = 200

#         if val_loader:
#             print('Initial evaluating on validation dataset')
#             val_loss, val_acc = self.measure_performance(val_loader)
#             epoch_summary = f'[Epoch 0] | Val acc: {val_acc:.4f} Val loss: {val_loss:.4f}\n\n'
#             print(epoch_summary)

#         for epoch in range(self.n_epochs):
#             print(f'--- Epoch: {epoch} ---')
#             epoch_start_time = time.time()
#             batch_start_time = time.time()
#             running_loss = 0.0

#             for i, data in enumerate(train_loader):
#                 input_ids = data[0].to(self.device)
#                 attn_masks = data[1].to(self.device)
#                 labels = data[2].to(self.device)

#                 self.optimizer.zero_grad()

#                 # Evaluation/optimization step
#                 outputs = self.model(input_ids=input_ids, attention_mask=attn_masks)[0]
#                 loss = self.criterion(outputs, labels)
#                 loss.backward()
#                 self.optimizer.step()
#                 if self.scheduler:
#                     self.scheduler.step()
                
#                 # Print statistics periodically
#                 running_loss += loss.item()
#                 if i % N_MINI_BATCH_CHECK == N_MINI_BATCH_CHECK - 1:
#                     batch_end_time = time.time()
#                     total_batch_time = batch_end_time - batch_start_time

#                     print(
#                         f'[E{epoch + 1:d} B{i + 1:d}] ',
#                         f'Loss: {running_loss / N_MINI_BATCH_CHECK:.5f} ',
#                         f'Time: {total_batch_time:.2f} ',
#                         f'LR: {self.scheduler.get_last_lr()}' if self.scheduler else '')

#                     # Reset statistics
#                     batch_start_time = time.time()
#                     running_loss = 0.0

#             epoch_end_time = time.time()
#             total_epoch_time = epoch_end_time - epoch_start_time
#             epoch_summary = '[Epoch {}] {} seconds'.format((epoch + 1), total_epoch_time)
            
#             if val_loader:
#                 val_loss, val_acc = self.measure_performance(val_loader)
#                 epoch_summary += f' | Val acc: {val_acc:.4f} | Val loss: {loss:.4f}'

#             print(epoch_summary)

#         print('Finished training')

#     def lr_test(self, train_loader, lrs=(-9, 2)):
#         """
#         lrs = (min_lr, max_lr, factor_scale)
#         """

#         # Prepare LR-finder loop
#         model = copy.deepcopy(self.model).to(self.device)
#         min_lr, max_lr = lrs
#         lrs = np.logspace(min_lr, max_lr, num=len(train_loader), endpoint=True)
#         losses = []
#         for i, data in enumerate(iter(train_loader)):
#             curr_lr = lrs[i]
#             optimizer = torch.optim.SGD(model.parameters(), lr=curr_lr)

#             input_ids = data[0].to(self.device)
#             attn_masks = data[1].to(self.device)
#             labels = data[2].to(self.device)

#             # Evaluation/optimization step
#             outputs = model(input_ids=input_ids, attention_mask=attn_masks)[0]
#             loss = self.criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             losses.append(loss)

#             if i % 100 == 0:
#                 print(f'Step [{i}, {len(train_loader)}] | LR: {curr_lr:.4e} | Loss: {loss:.4f}')

#         return losses

In [ ]:
# trainer = Trainer(model, 10, optimizer)

In [ ]:
# trainer.train_loop(train_loader, val_loader)